In [1]:
import requests
from bs4 import BeautifulSoup

### Step1. 첫번째 페이지만 크롤링 해 보기

In [2]:
url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=230&sid1=105&date=20190122&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [3]:
crawl_ls = soup.select('div[class="list_body newsflash_body"] ul[class="type06_headline"] li')    # 처음 10개 기사
crawl_ls += soup.select('div[class="list_body newsflash_body"] ul[class="type06"] li')    # 다음 10개 기사

In [4]:
url_ls = []
for crawl in crawl_ls: 
    url_ls += crawl.select('dt:nth-of-type(1) a') # 1번째 dt태그 지정

In [6]:
for url in url_ls:
    response = requests.get(url['href'])
    soup = BeautifulSoup(response.text,'html.parser')
    
    for match in soup.findAll('script'):
        match.replace_with('')
        
    content_ls = soup.select('div[id="articleBodyContents"]')
    for content in content_ls:
        print(content.get_text().strip())

배정회 신임 과학기술일자리진흥원장[아시아경제 이민우 기자] 배정회 전 국립중앙과학관전시연구단장(56)이 과학기술일자리진흥원 제3대 원장으로 취임했다.과학기술정보통신부는 22일 이 같은 인사를 발표했다. 임기는 이날부터 3년 동안이다.배 신임 원장은 행정고등고시 40회로 공직에 입문했다. 교육과학기술부 융합기술과장, 중앙대 경영경제대 초빙교수, 과기정통부연구성과활용정책과장 및 연구성과혁신기획과장 등을 거쳤다. 과학기술 정책 수립부터 정부 연구개발(R&D;), 연구성과의 기술 이전 및 사업화까지 두루 경험을 보유한 것으로 평가받는다.배 신임 원장은 이날 환담회에서 "새로운 임무에 맞게 조직을 신속히 재구성하고 직원 역량을 강화하겠다'며 "실험실 유망기술을 R&D; 전문기업과 연결하며 기술이전을 효과적으로 지원하고 신기술창업을 통한 미래형 일자리를 많이 창출시키겠다"고 밝혔다.이민우 기자 letzwin@asiae.co.kr▶ 네이버 홈에서 '아시아경제' 뉴스 확인하기▶ 재미와 신기 '과학을읽다' ▶ 꿀잼 '인기만화'<ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제 무단전재 배포금지>
량화 화웨이 의장 "해외 당국자 우리 시설 참관 환영"량화 화웨이 이사회 의장 <출처=화웨이> © 뉴스1(서울=뉴스1) 강민경 기자 = 중국 통신장비업체 화웨이가 22일(현지시간) 다보스포럼에서 자사 통신장비가 타국의 안보를 침해한다는 서방국들의 주장을 강하게 반박했다.로이터통신에 따르면, 량화(梁華) 화웨이 이사회 의장은 이날 스위스 다보스에서 열린 세계경제포럼(WEF)에서 취재진과 만나 "해외 당국자들이 요청한다면 우리 시설을 직접 참관하게 할 수 있다"면서 안보침해 의혹을 부정했다.량 의장은 "우리는 전 세계적으로 사업을 운영하고 있으며 모든 국가에서 현지 법률과 규제를 철저히 준수한다"면서 화웨이 제품 개발 시설과 여타 사업부에 대한 참관 요청을 환영한다고 강조했다.또 그는 화웨이가 지난 1일 캐나다에서 미국의 대(對)이란 제재 위반 혐의로 체포된 멍완저우(孟晩舟) 화웨이 부회장 

텐센트, 넷이즈 등 中 대형게임사도 미발급(지디넷코리아=김한준 기자)중국이 또 다시 신규 게임에 판호를 발급했다. 판호는 문화 콘텐츠를 대상으로 중국 국가신문출판광전총국(이하 신문출판국)이 발행하는 일종의 서비스 허가권이다.신문출판국은 22일 공식 홈페이지를 통해 판호 발급 목록을 공개했다. 올해 들어 두 번째 판호 발급이며, 지난해 12월 29일 판호 발급을 재개한 이후로는 세 번째다.이번에 판호를 발급 받은 게임은 총 93개로 플랫폼별로 살펴보면 모바일게임 89개, 웹게임 3개, 온라인게임 1개다.판호 발급 재개 이후 게임업계 관계자가 꾸준히 관심을 보이고 있는 한국 게임의 판호 발급 소식은 전해지지 않았다. 지금까지 257개의 게임에 대한 판호가 발급되는 와중에 한국 게임은 단 한 개도 심사를 통과하지 못 했다.한편 텐센트, 넷이즈 등 중국의 대형 게임사들 3차 판호 발급 명단에 이름을 올리지 못 했다.김한준 기자(khj1981@zdnet.co.kr) ▶ 지디넷코리아 '홈페이지' / ▶ 지디넷코리아 ‘이슈진단+’  ▶ 네이버 채널에서 ‘지디넷코리아’를 구독해주세요
KT스카이라이프 "세부 내용은 확인 필요"국회 과방위, 합산규제 문제 2월 재논의【서울=뉴시스】이국현 최선윤 기자 = 국회 과학기술정보방송통신위원회가 KT스카이라이프의 공공성 회복을 위해 계열분리 등 KT로부터의 독립 방안을 논의한 것으로 알려졌다.이에 KT스카이라이프는 22일 "논란 중인 계열분리 등 세부내용은 법안소위에 참여한 의원실과 과학기술정보통신부에 확인이 필요하다"며 "현재 확인한 내용은 과기정통부에서 합산규제 관련 방안을 준비해 2월 임시국회에서 재논의 한다는 것 뿐"이라고 밝혔다.이날 국회 과방위는 국회에서 정보통신방송법안심사소위원회를 열고 유료방송 합산규제 재도입 여부를 논의한 뒤 2월 임시국회에서 이를 다시 논의키로 결론냈다.합산규제란 IPTV(인터넷TV), 케이블TV, 위성방송 등 유료방송 시장에서 특정 사업자가 전체 시장점유율의 3분의 1(33%)을 넘지 못하도록 한 제도다

[일간스포츠 권오용]  유영민 과학기술정보통신부 장관(우측)과 하현회 LG유플러스 부회장이 5G 클라우드 VR게임을 체험하고 있는 모습. LGU+ 제공 중국 통신장비 업체 화웨이와 손잡아 우려를 낳고 있는 LG유플러스는 국내 기업들과 함께 5G 생태계 선순환을 만들어가겠다고 밝혔다. LG유플러스는 22일 유영민 과학기술정보통신부 장관, 성윤모 산업통상자원부 장관, 홍종학 중소벤처기업부 장관 등 3개 부처 장관이 5G 현장 방문을 위해 서울시 강서구 LG사이언스파크 내 LG유플러스 마곡사옥을 방문한 자리에서 국내 기업과의 상생을 강조했다. 이날 행사에는 LG유플러스 하현회 부회장, LG전자 정도현 사장, LG사이언스파크 안승권 사장 등을 비롯해 삼지전자, 유비쿼스, 우성엠엔피, 삼화콘덴서 등 LG유플러스 및 LG전자 협력회사 대표 8명이 참석했다.LG유플러스는 2019년 상반기 서울, 수도권 및 광역시를 시작으로 2019년 말까지 주요 지역에 5G 네트워크 구축을 완료하고, 이후 85개시·군 지역으로 점차 확대해 나갈 계획이라고 밝혔다.LG유플러스와 LG전자는 대중소 상생협력 프로그램도 소개했다. LG유플러스는 국내 중소기업에게 연구개발비와 기술인력 등을 지원해 장비를 국산화하는 프로그램을 운영 중이다.LG유플러스는 가상화 통합관리 플랫폼(NFV MANO) 연동규격을 중소협력사에게 공개하고 첫 사례로 중소 통신장비회사인 아리아텍과 함께 가상화 기반 5G 가입자 인증 및 정책 관리 장비를 개발, 연내 상용화하기로 했다.가상화 통합관리 플랫폼은 5G 서비스를 제공하기 위해 필요한 다양한 코어 장비들을 필요한 시점에 네트워크에 바로 적용할 수 있도록 해준다.아리아텍과 함께 개발한 가입자 인증 및 정책 관리 장비는 5G 단말을 가진 고객이 국내 또는 해외에서 5G 네트워크에 접속 시, 가입한 요금제와 부가서비스에 따라 서비스별로 최적의 품질을 보장해준다.또 LG유플러스는 유비쿼스와 100Gbps 스위치 개발을 완료하고, 이후 수십억원 상당의 자금을 직접 지원해 안정적으

KeyboardInterrupt: 

### Step2. 함수화하여 5페이지 크롤링하기

In [5]:
def get_url(page):
        url = 'https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=230&sid1=105&date=20190122&page='
        url += str(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')

        crawl_ls = soup.select('div[class="list_body newsflash_body"] ul[class="type06_headline"] li')    # 처음 10개 기사
        crawl_ls += soup.select('div[class="list_body newsflash_body"] ul[class="type06"] li')    # 다음 10개 기사

        url_ls = []
        for crawl in crawl_ls: 
            url_ls += crawl.select('dt:nth-of-type(1) a')    # 1번째 dt태그 지정

        return url_ls

In [7]:
def get_content(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')

        # script,br태그 지우기
        for match in soup.findAll('script'):
            match.replace_with('')
        for match in soup.findAll('br'):
            match.replace_with('')

        content_ls = soup.select('div[id="articleBodyContents"]')

        for content in content_ls:
            f.write(content.get_text().strip())

In [ ]:
# 파일 열기
f = open('IT_news.csv','w',encoding='utf-8')

# 5페이지까지 크롤링
for page in [1,2,3,4,5]:
    url_ls = get_url(page)
    for url in url_ls:
        get_content(url['href'])

# 파일 닫기
f.close()